# Invader Defender 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog

In [2]:
actions = [[-1, 0], [0, 1], [1, 0], [0, -1]] #up, right, down, left = (clockwise from up) 
action_count = len(actions) 
gridSize = 6 
state_count = gridSize*gridSize

In [3]:
class Invader_Defender():
    def __init__(self, gridSize):
        self.valueMap = np.zeros((gridSize, gridSize))
        self.states = [[i, j] for i in range(gridSize) for j in range(gridSize)]
        self.size = gridSize
        
        # deterministic transition ?
        self.transition_prob = 1 
        
        # initialize defender and invader states
        self.new_state = [0, 0, 0, 0]
        self.new_defender_state = [0, 0]
        self.new_invader_state = [0, 0]
        
        # set territory state
        self.territory_state = [4, 4]

        # create a list of all possible states in the game
        self.game_state_list = []
        for defender_state in self.states:
            for invader_state in self.states:
                combined_states = defender_state + invader_state
                self.game_state_list.append(combined_states)
        
        # create 2 lists of states representing defender and invader victory
        self.defender_won = []
        self.invader_won = []
        
        # create states representing defender victory
        for defender_state in self.states:
            for invader_state in self.states:
                distance = np.linalg.norm(np.array(defender_state) - np.array(invader_state))
                # if the invader is not at territory and within the capture range of defender = defender won
                if invader_state != self.territory_state and distance <= np.sqrt(2):
                    combined_states = defender_state + invader_state
                    self.defender_won.append(combined_states)
           
        # create states representing invader victory
        for defender_state in self.states:
            distance = np.linalg.norm(np.array(defender_state) - np.array(self.territory_state))
            # if the invader is at territory, and outside of the defender's capture range = invader won
            if distance > np.sqrt(2):
                combined_states = defender_state + self.territory_state
                self.invader_won.append(combined_states)
    
    def possible_states(self):
        """
        A function that returns a list of all possible states in the game
        """
        return self.game_state_list
    
    def terminal_check(self, state):
        """
        A function that checks whether the game is at a terminal state.
        Terminal state happens when either the invader or defender has won.
        """
        if state in self.defender_won:
            status = "Defender Won"
            terminal_check = True
        elif state in self.invader_won:
            status = "Invader Won"
            terminal_check = True
        else:
            terminal_check = False
            status = "Game in Progress"

        return terminal_check, status
    
#     def transition_probability(self, transition):
#         """
#         A function that returns the transition probability...?
#         """
#         return self.transition_prob, reward

    def next_state(self, current_state, defender_action, invader_action):
        """
        A function that returns the next state
        Input: current state [0,0] , defender_action [0, 1], invader_action [0,-1]
        Output: next state array([x1,y1,x2,y2]) and reward (int)
            - If the action takes the agent off grid, the agent remains in original state
            - If defender won, reward is calculated based on manhattan distance between invader captured state
            and territory
            - If defender loss, reward is -100
        """
        defender_state = []
        invader_state = []
        
        # deconstruct current state [0,0,1,1] in to defender [0,0] and invader [1,1] state
        for i in range(4):
            if i < 2:
                defender_state.append(current_state[i])
            else:
                invader_state.append(current_state[i])
                
        # get next state: state: [0, 0], action: [0, 1], new_state = [0, 1]
        self.new_defender_state = list(np.array(defender_state) + np.array(defender_action))
        self.new_invader_state = list(np.array(invader_state) + np.array(invader_action))

        # if new defender states results in off the grid, return to original state
        if -1 in self.new_defender_state or self.size in self.new_defender_state:
            self.new_defender_state = defender_state
        
        # if new invader states results in off the grid, return to original state
        if -1 in self.new_invader_state or self.size in self.new_invader_state:
            self.new_invader_state = invader_state
       
        # combine the defender and invader state
        self.new_state = self.new_defender_state
        self.new_state.extend(self.new_invader_state)
        
        # calculate rewards
        terminal, status = self.terminal_check(self.new_state)
        if terminal == True:
            if status == "Defender Won":
                # defender reward if defender won (manhattan distance between invader captured state and territory)
                distance_to_territory = sum(abs(np.array(self.new_invader_state) - np.array(self.territory_state)))
                self.reward = distance_to_territory
            else:
                # defender reward if invader won
                self.reward = -100
        else:
            self.reward = 0
            
        return self.new_state, self.reward

## Testing 

In [4]:
invader_defender = Invader_Defender(6)

In [5]:
next_state, reward = invader_defender.next_state([2,1,0,0], [-1, 0], [-1, 0])

In [6]:
next_state

[1, 1, 0, 0]

In [7]:
reward

8

In [8]:
invader_defender.terminal_check([1, 1, 0, 0])

(True, 'Defender Won')

## Value Iteration 

In [9]:
invader_defender = Invader_Defender(6)

In [10]:
k = 0
U = {}
gamma = 0.9
state_list = []
listofzeros = [0.0] * len(invader_defender.game_state_list)

# convert game_state_list in to a state list of tuples in order to make a dictionary
for state in invader_defender.game_state_list:
    state_list.append(tuple(state))
    
# initiate params
G = dict(zip(state_list, listofzeros))
U[k] = dict(zip(state_list, listofzeros))

In [11]:
def calculate_payoff(state):
    """
    A function calculates the payoff of a specific state by iterating over every defender/invader action
    Input: state (ie. [0,0,1,1])
    Output: payoff = 4x4 matrix where each element represent the defender's payoff 
    when defender take i, and invader take action j
    """
    payoff = np.zeros([4,4])
    for i in range(action_count):
        defender_action = actions[i]
        for j in range(action_count):
            invader_action = actions[j]
            next_state, reward = invader_defender.next_state(state, defender_action, invader_action)
            payoff[i, j] = reward + gamma*invader_defender.transition_prob*U[k][tuple(next_state)]
    return payoff

In [12]:
def calculate_value(G_state):
    
    # defender lin prog
    c = [0, 0, 0, 0, -1]
    defender_q = -1*np.transpose(G_state)     
    v_coeff = np.ones((4,1))
    Aub = np.concatenate((defender_q,v_coeff),1)
    b = [0, 0, 0, 0]
    Aeq = [[1, 1, 1, 1, 0]]
    beq = [[1.]]
    bounds = ((0,1),(0,1),(0,1),(0,1),(None, None))
    defender_solution = linprog(c, A_ub=Aub, b_ub=b, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')
    
    # invader lin prog
    c = [0, 0, 0, 0, 1]
    invader_q = G_state
    w_coeff = np.ones((4,1))*-1
    Aub = np.concatenate((invader_q,w_coeff),1)
    invader_solution = linprog(c, A_ub=Aub, b_ub=b, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')
    
    defender_value = defender_solution['fun']
    invader_value = invader_solution['fun']
    
    return defender_value, invader_value

In [13]:
# to remove warnings
import warnings
warnings.filterwarnings('ignore')

for state in invader_defender.game_state_list:
    
    # Build G dictionary {state: payoff (4x4)}
    G[tuple(state)] = calculate_payoff(state)
    defender_value, invader_value = calculate_value(G[tuple(state)])
    # calculate value of game
    print("state: ", state)
    print("defender value: ", defender_value)
    print("invader value: ", invader_value)
    print("---------------------")
    
    # calculate delta
    
    # if delta small enough, break
    
    

state:  [0, 0, 0, 0]
defender value:  -7.0
invader value:  7.0
---------------------
state:  [0, 0, 0, 1]
defender value:  -6.0
invader value:  6.0
---------------------
state:  [0, 0, 0, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 0, 0, 3]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 0, 0, 4]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 0, 0, 5]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 0, 1, 0]
defender value:  -6.0
invader value:  6.0
---------------------
state:  [0, 0, 1, 1]
defender value:  -2.5
invader value:  2.5
---------------------
state:  [0, 0, 1, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 0, 1, 3]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 0, 1, 4]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 0, 1, 5]
defender value:  0.0
invader value:  0.0
----------

state:  [0, 2, 5, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 2, 5, 3]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 2, 5, 4]
defender value:  100.0
invader value:  -100.0
---------------------
state:  [0, 2, 5, 5]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 3, 0, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 3, 0, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 3, 0, 2]
defender value:  -5.0
invader value:  5.0
---------------------
state:  [0, 3, 0, 3]
defender value:  -4.0
invader value:  4.0
---------------------
state:  [0, 3, 0, 4]
defender value:  -3.0
invader value:  2.9999999999999996
---------------------
state:  [0, 3, 0, 5]
defender value:  -4.0
invader value:  4.0
---------------------
state:  [0, 3, 1, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 3, 1, 1]
defender value:  0.0
invader va

state:  [0, 5, 3, 5]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 5, 4, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 5, 4, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 5, 4, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 5, 4, 3]
defender value:  100.0
invader value:  -100.0
---------------------
state:  [0, 5, 4, 4]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 5, 4, 5]
defender value:  100.0
invader value:  -100.0
---------------------
state:  [0, 5, 5, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 5, 5, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 5, 5, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 5, 5, 3]
defender value:  0.0
invader value:  0.0
---------------------
state:  [0, 5, 5, 4]
defender value:  100.0
invader value:  -100.0

state:  [1, 2, 2, 3]
defender value:  -1.0
invader value:  1.0
---------------------
state:  [1, 2, 2, 4]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 2, 2, 5]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 2, 3, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 2, 3, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 2, 3, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 2, 3, 3]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 2, 3, 4]
defender value:  99.99999999999999
invader value:  -99.99999999999991
---------------------
state:  [1, 2, 3, 5]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 2, 4, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 2, 4, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 2, 4, 2]
defender value:  0.0
inva

state:  [1, 5, 0, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 5, 0, 3]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 5, 0, 4]
defender value:  -3.0
invader value:  2.9999999999999996
---------------------
state:  [1, 5, 0, 5]
defender value:  -4.0
invader value:  4.0
---------------------
state:  [1, 5, 1, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 5, 1, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 5, 1, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 5, 1, 3]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 5, 1, 4]
defender value:  -2.0
invader value:  2.0
---------------------
state:  [1, 5, 1, 5]
defender value:  -3.0
invader value:  3.0
---------------------
state:  [1, 5, 2, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [1, 5, 2, 1]
defender value:  0.0
invader value: 

...

state:  [3, 4, 0, 5]
defender value:  0.0
invader value:  0.0
---------------------
state:  [3, 4, 1, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [3, 4, 1, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [3, 4, 1, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [3, 4, 1, 3]
defender value:  0.0
invader value:  0.0
---------------------
state:  [3, 4, 1, 4]
defender value:  0.0
invader value:  0.0
---------------------
state:  [3, 4, 1, 5]
defender value:  0.0
invader value:  0.0
---------------------
state:  [3, 4, 2, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [3, 4, 2, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [3, 4, 2, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [3, 4, 2, 3]
defender value:  -2.0
invader value:  2.0
---------------------
state:  [3, 4, 2, 4]
defender value:  -1.0
invader value:  1.0
------------

state:  [4, 1, 0, 4]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 1, 0, 5]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 1, 1, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 1, 1, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 1, 1, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 1, 1, 3]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 1, 1, 4]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 1, 1, 5]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 1, 2, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 1, 2, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 1, 2, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 1, 2, 3]
defender value:  0.0
invader value:  0.0
--------------

state:  [4, 3, 5, 4]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 3, 5, 5]
defender value:  -1.0
invader value:  1.0
---------------------
state:  [4, 4, 0, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 4, 0, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 4, 0, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 4, 0, 3]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 4, 0, 4]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 4, 0, 5]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 4, 1, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 4, 1, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 4, 1, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [4, 4, 1, 3]
defender value:  0.0
invader value:  0.0
-------------

state:  [5, 1, 0, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 1, 0, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 1, 0, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 1, 0, 3]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 1, 0, 4]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 1, 0, 5]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 1, 1, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 1, 1, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 1, 1, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 1, 1, 3]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 1, 1, 4]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 1, 1, 5]
defender value:  0.0
invader value:  0.0
--------------

state:  [5, 3, 5, 2]
defender value:  -2.0
invader value:  2.0
---------------------
state:  [5, 3, 5, 3]
defender value:  -1.0
invader value:  1.0
---------------------
state:  [5, 3, 5, 4]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 3, 5, 5]
defender value:  -1.0
invader value:  1.0
---------------------
state:  [5, 4, 0, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 4, 0, 1]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 4, 0, 2]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 4, 0, 3]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 4, 0, 4]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 4, 0, 5]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 4, 1, 0]
defender value:  0.0
invader value:  0.0
---------------------
state:  [5, 4, 1, 1]
defender value:  0.0
invader value:  0.0
-----------

In [14]:
G[tuple(state)]

array([[1., 2., 2., 1.],
       [1., 2., 2., 1.],
       [1., 2., 2., 1.],
       [1., 2., 2., 1.]])